In [1]:
# Добавляем директорию для равномерного импорта всего с корневой локации
import sys

sys.path.append('../')

# Импортируем библиотеку, которая автоматически настраивает коннектор для системы выполнения кода
from src.libs.LCTWrapTwin import start_module

In [2]:
# ЭТОТ БЛОК КОДА МОЖНО И НУЖНО МОДИФИЦИРОВАТЬ
# Как обычно, подсказки - в импортах

from typing import Literal
from src.libs.LCTWrapTwin.Modules.Handler import MissionHandler, TrustedHandler
import socket
import threading
import time
from math import sqrt

class UserMissionHandler(MissionHandler):

    @staticmethod
    def config_cyber_obstacles() -> dict:

        return {
            "CybP_01": False,  # (Компрометация кода АП) - хэш автопилота
            "CybP_02": False,  # (неполадки связи = смешное сообщение)
            "CybP_03": False,  # привода, поддел инфы о них) - если взлом, то 'eeeeeeee'
            "CybP_04": False,  # (компроментация навиг. системы) - если взлом, то выдает неверны данные
            "CybP_05": False,  # компроментация шлагбаума
            "CybP_06": False,  # постоянно (несанкц. ускорение)
        }

    def get_photo(self):  # Для отладки компьютерного зрения [debug] #
        from datetime import datetime
        import os
        import cv2
        import numpy as np

        frame = super().get_camera_frame()
        
        if frame is not None:
            save_dir = "../../Photos"

            # Генерируем уникальное имя файла с временной меткой
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")[:-3]  # миллисекунды
            
            # Сохраняем изображение
            image_filename = os.path.join(save_dir, f"frame_{timestamp}.jpg")
            cv2.imwrite(image_filename, frame)
            self.lg.log(f"Сохранение изображения")
            
            # Текстовый файл (только цвета пикселей)
            array_filename = os.path.join(save_dir, f"frame_{timestamp}.txt")
            np.savetxt(array_filename, frame.reshape(-1, frame.shape[-1]), fmt='%d', delimiter=',')
            self.lg.log(f"Сохранение массива")
        
        return frame


        
        return True  # пока что всегда возвращает True

    def check_t_light_CV(self) -> int:  # Проверка светофора на зеленый/красный [КЗ] #

        velocity = 20
        green = False
        red = False
        self.lg.log("🚦 Внимание! Светофор 🚦")

        while True:
            frame = self.get_camera_frame()
            center_row = frame[240, :]
            
            green_pixels = 0
            red_pixels = 0
            
            for x in range(len(center_row)):
                pixel = center_row[x]
                b, g, r = pixel[0].astype(int), pixel[1].astype(int), pixel[2].astype(int)
                
                is_green = (b in range(40-velocity, 40+velocity)) and (g in range(200-velocity, 200+velocity)) and (r in range(40-velocity, 40+velocity))
                
                is_red = (b in range(40-velocity, 40+velocity)) and (g in range(40-velocity, 40+velocity)) and (r in range(200-velocity, 200+velocity))
                
                if is_green:
                    green_pixels += 1      
                elif is_red:
                    red_pixels += 1
                
            if green_pixels > red_pixels:
                if not green:
                    green = True
                    self.lg.log("🟢 Светофор ЗЕЛЁНЫЙ. Вперед! 🟢")
                break
            else:
                if not red:
                    red = True
                    self.lg.log("🔴 Светофор КРАСНЫЙ. Ожидаем... 🔴")
                time.sleep(1.0)

        return True 

    def check_pedestrian_CV(self) -> bool:  # Проверка положения пешехода [КЗ] #
        BGR_TARGET_1 = (188, 193, 196)  # нвоые значения bgr
        x_1, y_1 = 11, 309
        x_2, y_2 = 507, 352
        x_3, y_3 = 490, 390
        x_4, y_4 = 3, 317
        velocity = 20
        wait = False
        
        while True:
            frame = self.get_camera_frame()
            pixels_1 = frame[y_1, x_1-10:x_1+10]
            pixels_2 = frame[y_2, x_2-5:x_2+5]
            pixels_3 = frame[y_3, x_3-5:x_3+5]
            pixels_4 = frame[y_4, x_4-5:x_4+5]
            
            mas_1 = [[pix[i] in range(BGR_TARGET_1[i] - velocity, BGR_TARGET_1[i] + velocity) for i in range(3)] for pix in pixels_1]
            mas_2 = [[pix[i] in range(BGR_TARGET_1[i] - velocity, BGR_TARGET_1[i] + velocity) for i in range(3)] for pix in pixels_2]
            mas_3 = [[pix[i] in range(BGR_TARGET_1[i] - velocity, BGR_TARGET_1[i] + velocity) for i in range(3)] for pix in pixels_3]
            mas_4 = [[pix[i] in range(BGR_TARGET_1[i] - velocity, BGR_TARGET_1[i] + velocity) for i in range(3)] for pix in pixels_4]
            
            result_1 = any([all(i) for i in mas_1])
            result_2 = any([all(i) for i in mas_2])
            result_3 = any([all(i) for i in mas_3])
            result_4 = any([all(i) for i in mas_4])
            
            if result_1 or result_2 or result_3:
                self.lg.log(f"🚶‍♂️Пешеход перешел дорогу. Вперед!🚶‍♂️")
                break
            else:
                if not wait:
                    wait = True
                    self.lg.log(f"🚸 Пешеходный переход. Ожидаем... 🚸")
                continue
        return True

    def change_barrier_CV(self) -> int:  # Смена положения шлагбаума [КЗ] #
        
        BGR_TARGET_1 = [44, 43, 117]  # без тени
        BGR_TARGET_2 = [50, 50, 50]  # с учетом тени
        x, y = 301, 273
        velocity = 30

        def check_status():
            frame = self.get_camera_frame()
            pixels = frame[y, x-20:x+20]
            for pixel in pixels:
                mass_1 = [pixel[i] in range(BGR_TARGET_1[i] - velocity, BGR_TARGET_1[i] + velocity) for i in range(3)]
                mass_2 = [pixel[i] in range(BGR_TARGET_2[i] - velocity, BGR_TARGET_2[i] + velocity) for i in range(3)]
                if all(mass_1) or all(mass_2):
                    return 0  # закрыт
            return 1  # открыт
        
        start_status = check_status()
        repeated_failure = False
        if not start_status:
            while True:
                self.set_barrier_toggle()
                time.sleep(2.0)
                if check_status():
                    self.lg.log("✅ Шлагбаум ОТКРЫТ. Можно продолжать движение. ✅")
                    break
                else:
                    if not repeated_failure:
                        self.lg.log("❌ Устройство переключения шлагбаума работает нестабильно ❌")
                        repeated_failure = True
                    else:
                        self.lg.log("   ❌🎲 ПОВТОРНОЕ не срабатывание устройства переключения 🎲❌")
        else:
            self.lg.log("✅ Шлагбаум ОТКРЫТ. Можно продолжать движение. ✅")

    def mission_code(self):
        
        def robot_marshrut():
            self.set_robot_speed(0.24)
            self.set_brush_speed(100)
            # self.change_barrier_UDP("drop")  # Сразу закрываем шлагбаум дистанционно (уже так не сделать)
            self.ap_hook.do_rotate({"x": 0.6, "y": 1})
            self.ap_hook.do_move({"x": 0.6, "y": 1})
            self.ap_hook.do_rotate({"x": 0.68, "y": 1.4})
            self.ap_hook.do_move({"x": 0.68, "y": 1.4})  # Шлагбаум 
            self.change_barrier_CV()

            self.ap_hook.do_move({"x": 0.68, "y": 2.25})  # Пешеход верхний
            self.check_pedestrian_CV()
            # self.change_barrier_UDP("drop")  # Снова закрываем шлагбаум дистанционно (уже так не сделать)

            self.ap_hook.do_move({"x": 0.68, "y": 2.92})
            self.ap_hook.do_rotate({"x": 1, "y":2.92})
            self.ap_hook.do_move({"x": 1, "y": 2.92})  # Возможное столкновение с роботом (первый чек)
            # self.check_autobot_UDP()  # Пока просто првоеряет на компроментацию МТС
            
            self.ap_hook.do_move({"x": 2.28, "y":2.92})
            self.ap_hook.do_rotate({"x": 2.28, "y":3.32})
            self.ap_hook.do_move({"x": 2.28, "y":3.32})
            self.ap_hook.do_rotate({"x": 2.6, "y":3.32})
            self.ap_hook.do_move({"x": 2.7, "y":3.32})
            self.ap_hook.do_rotate({"x": 2.6, "y":3.4})
            self.ap_hook.do_move({"x": 2.6, "y":3.4})
            self.ap_hook.do_move({"x": 2.16, "y":3.4})
            self.ap_hook.do_rotate({"x": 2.12, "y":3.08})
            self.ap_hook.do_move({"x": 2.12, "y":3.08})
            self.ap_hook.do_rotate({"x": 1.8, "y":3.08})
            self.ap_hook.do_move({"x": 1.6, "y":3.08}) #
            self.ap_hook.do_rotate({"x": 1.32, "y":2.12}) #
            self.ap_hook.do_move({"x": 1.32, "y":2.12}) #

            self.ap_hook.do_rotate({"x": 1.32, "y":2.12})
            self.ap_hook.do_move({"x": 1.32, "y":2.12})
            self.ap_hook.do_rotate({"x": 1.8, "y":2.12})
            self.ap_hook.do_move({"x": 1.8, "y":2.12})
            self.ap_hook.do_move({"x": 2.6, "y":2.12})
            self.ap_hook.do_rotate({"x": 2.65, "y":2.2})
            self.ap_hook.do_move({"x": 2.65, "y":2.2})
            self.ap_hook.do_rotate({"x": 2.6, "y":2.28})
            self.ap_hook.do_move({"x": 2.6, "y":2.28})
            self.ap_hook.do_rotate({"x": 1.2, "y":2.28})
            self.ap_hook.do_move({"x": 1.8, "y":2.28})  # Светофор 1 (центральный)
            self.check_t_light_CV()

            self.ap_hook.do_move({"x": 1.36, "y":2.28})
            self.ap_hook.do_rotate({"x": 1.32, "y":0.8})
            self.ap_hook.do_move({"x": 1.32, "y":1})  # Светофор 0 (нижний)
            self.check_t_light_CV()

            self.ap_hook.do_move({"x": 1.32, "y":0.8})
            self.ap_hook.do_rotate({"x": 1.4, "y":0.6})
            self.ap_hook.do_move({"x": 1.4, "y":0.6})
            self.ap_hook.do_rotate({"x": 1.48, "y":1})
            self.ap_hook.do_move({"x": 1.48, "y":1})
            self.ap_hook.do_move({"x": 1.48, "y":1.4})
            self.ap_hook.do_move({"x": 1.48, "y":1.8})
            self.ap_hook.do_move({"x": 1.48, "y":2.2})
            self.ap_hook.do_move({"x": 1.48, "y":2.6})  # Возможное столкновение с роботом (второй чек)
            # self.check_autobot_UDP()  # Пока просто првоеряет на компроментацию МТС

            self.ap_hook.do_move({"x": 1.48, "y":3.08})
            self.ap_hook.do_rotate({"x": 0.52, "y":3.08})
            self.ap_hook.do_move({"x": 0.52, "y":3.08})
            self.ap_hook.do_rotate({"x": 0.52, "y":3.08})
            self.ap_hook.do_move({"x": 0.52, "y":3.08})
            self.ap_hook.do_rotate({"x": 0.52, "y":2.85})
            self.ap_hook.do_move({"x": 0.52, "y":2.95})  # Верхний пешеход
            self.check_pedestrian_CV()

            self.ap_hook.do_move({"x": 0.52, "y":2.2})  # Шлагбаум 
            self.change_barrier_CV()

            self.ap_hook.do_move({"x": 0.52, "y":1.8})
            self.ap_hook.do_move({"x": 0.52, "y":1.4})
            self.ap_hook.do_move({"x": 0.52, "y":1})
            self.ap_hook.do_rotate({"x": 0.52, "y":0.52})
            self.ap_hook.do_move({"x": 0.52, "y":0.52})
            self.ap_hook.do_rotate({"x": 1, "y":0.52})
            self.ap_hook.do_move({"x": 1, "y":0.52})

            self.ap_hook.do_move({"x": 1.85, "y":0.52})  # Пешеход нижний 
            self.check_pedestrian_CV()

            self.ap_hook.do_move({"x": 2.2, "y":0.52})
            self.ap_hook.do_move({"x": 2.6, "y":0.52})    
            self.ap_hook.do_rotate({"x": 3.08, "y":0.52})
            self.ap_hook.do_move({"x": 3.08, "y":0.52})
            self.ap_hook.do_rotate({"x": 3.08, "y":1})  # Возможное столкновение с роботом (третий чек)
            # self.check_autobot_UDP()  # Пока просто првоеряет на компроментацию МТС

            self.ap_hook.do_move({"x": 3.08, "y":1})
            self.ap_hook.do_move({"x": 3.08, "y":1.4})
            self.ap_hook.do_move({"x": 3.08, "y":1.8})
            self.ap_hook.do_move({"x": 3.08, "y":2.2})
            self.ap_hook.do_move({"x": 3.08, "y":2.6})
            self.ap_hook.do_move({"x": 3.08, "y":3.0})
            # self.check_autobot_UDP() !!! НЕ ЗДЕСЬ !!!
            self.ap_hook.do_rotate({"x": 3, "y":3.4})
            self.ap_hook.do_move({"x": 3.0, "y":3.4})
            self.ap_hook.do_rotate({"x": 2.92, "y":3})
            self.ap_hook.do_move({"x": 2.92, "y":3})
            self.ap_hook.do_rotate({"x": 2.92, "y":1.0})
            self.ap_hook.do_move({"x": 2.92, "y":1.0})
            self.ap_hook.do_move({"x": 2.92, "y":0.68})
            self.ap_hook.do_rotate({"x": 2.6, "y":0.68})
            self.ap_hook.do_move({"x": 2.6, "y":0.68})
            self.ap_hook.do_move({"x": 2.45, "y":0.68})  # Пешеход нижний
            self.check_pedestrian_CV()

            self.ap_hook.do_move({"x": 1.8, "y":0.68})
            self.ap_hook.do_move({"x": 1.4, "y":0.68}) 
            self.ap_hook.do_move({"x": 1.0, "y":0.68})
            self.ap_hook.do_rotate({"x": 0.68, "y":0.68})
            self.ap_hook.do_move({"x": 0.68, "y":0.68})
            self.ap_hook.do_rotate({"x": 0.6, "y":1.0})
            self.ap_hook.do_move({"x": 0.6, "y":1.0})
            self.ap_hook.do_rotate({"x": 0.2, "y":1.0})
            self.ap_hook.do_move({"x": 0.2, "y":1.0})
            self.ap_hook.do_rotate({"x": 0.6, "y":1.0}) 

        robot_marshrut()



class UserTrustedHandler(TrustedHandler):
    
    def __init__(self, context):
        super().__init__(context)

    def trusted_code(self):
        
        self._check_robot_speed_and_position()

    def make_next_short_message(self, prev_message: str):
        
        self.cybp02_message_count += 1
        self.cybp02_protection_active = True
        
        self._detect_cybp02_attack(prev_message)
        
        return self.last_correct_message

    def check_autobot_UDP(self) -> bool:  # Проверка компроментации МТС (CybP_06) [UDP] #
        
        def intercept_UDP(name: str) -> int:  # Перехват трафика умных устройств [UDP] #
            try:
                ports = {
                "traffic_0": 5041,  # светофор 0 (самый ближний)
                "traffic_1": 5042,  # светофр 1 (средний)
                "traffic_2": 5043,  # светофор 2 (самый дальний)
                "barrier": 5031,   # шлагбаум (больше не работает)
                "autobot": 5071    # автоматичский робот
                }

            except Exception as e:
                self.lg.log(f"Переданный в параметр name аргумент: {name}!")

            listen_port = ports[name]
            result = None
                
            interceptor_socket = None
            
            try:
                interceptor_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
                interceptor_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
                interceptor_socket.bind(('127.0.0.1', listen_port))
                
                try:
                    data = interceptor_socket.recvfrom(1024)[0]
                    decoded_data = data.decode('utf-8', errors='ignore').strip()
                    if name in tuple(ports.keys())[:-2]:  # светофоры
                        color = int(decoded_data[-2])
                        self.lg.log(f"{color}")
                        self.lg.log(f"{name} КРАСНЫЙ") if color == 1 else self.lg.log(f"{name} ЗЕЛЁНЫЙ")
                        if color == 2:
                            result = True
                    elif name in tuple(ports.keys())[-2]:  # шлагбаум
                        status = decoded_data[-2]
                        result = int(status)
                    elif name in tuple(ports.keys())[-1]:  # автоматичский робот
                        result = decoded_data

                    interceptor_socket.close()
                    return result
                    
                except Exception as e:
                    self.lg.log(f"Ошибка при получении данных: {e}")
                    
            except Exception as e:
                self.lg.log(f"Ошибка создания сокета: {e}")

        data = intercept_UDP("autobot")
        result = int(data[-2])
        repeated_failure = False
        if result:
            if not repeated_failure:
                repeated_failure = True
                self.lg.log('[📡<OP>🛜]::🛑 Обнаружена компроментация МТС! 🛑')
        else:
            repeated_failure = False
            self.lg.log('[📡<OP>🛜]::🍀 Компроментации МТС не обнаружено. 🍀')
        
        return True  # пока что всегда возвращает True  

### 
         
    def _check_robot_speed_and_position(self):
            
        def monitor():
            
            robot_status = self.get_robot_status()
            if robot_status:
                current_x = robot_status["position_x"]
                current_y = robot_status["position_y"]
                rotation = robot_status["rotation"]
                

                self._calculate_cell_from_position(current_x, current_y, int(rotation))

        while True:
            monitor_start = threading.Thread(target=monitor, daemon=True)
            monitor_start.start()

    def _calculate_cell_from_position(self, x, y, rotation):
                
        cell_x = x // 0.4 + 1
        cell_y = 9 - (y // 0.4 + 1)
        
        cell_number = int(cell_y * 9 + cell_x)

        # МТС
        if cell_number in (12, 22, 71): 
            self.send_message_to_ap({"type": "my_coords", "coords": self.get_robot_status()})

        # # ШЛАГБАУМ
        # elif (cell_number == 47 and rotation in range(80, 91)) \
        #     or (cell_number == 29 and rotation in range(-90, -87)) \
        #         and not self.check_barrier: 
        #     if not self.check_barrier:
        #         self.check_barrier = True 
        #         self.check_CybP_05_CV()
        
        if cell_number not in (29, 47):  
            self.check_barrier = False

        return cell_number


In [ ]:
start_module(UserMissionHandler, UserTrustedHandler)

[23:48]::INIT::AGTS-WRAP-CLIENT, версия 2.1.0
[23:48]::INIT::UDP(c) <- (uid=KHpl) инициализирован.
[23:48]::INIT::UDP(c) <- (uid=NYZk) инициализирован.
[23:48]::INIT::Загрузка завершена.
[23:48]::INIT::UDP(c) -> (uid=XBbi) инициализирован.
[23:48]::INIT::Адрес сервера (command_receiver): http://127.0.0.1:13500
[23:48]::INFO::(AP) Заезд инициализирован - ожидание старта
[23:48]::INFO::(TM) Заезд инициализирован - ожидание старта
[23:48]::INFO::Доверенный код инициализирован[23:48]::INFO::Код заезда инициализирован

[23:48]::INFO::❌ Устройство переключения шлагбаума работает нестабильно ❌
[23:48]::INFO::✅ Шлагбаум ОТКРЫТ. Можно продолжать движение. ✅
[23:48]::INFO::🚸 Пешеходный переход. Ожидаем... 🚸
[23:49]::INFO::🚶‍♂️Пешеход перешел дорогу. Вперед!🚶‍♂️
[23:51]::INFO::🚦 Внимание! Светофор 🚦
[23:51]::INFO::🟢 Светофор ЗЕЛЁНЫЙ. Вперед! 🟢
[23:51]::INFO::🚦 Внимание! Светофор 🚦
[23:51]::INFO::🟢 Светофор ЗЕЛЁНЫЙ. Вперед! 🟢
[23:51]::INFO::🚶‍♂️Пешеход перешел дорогу. Вперед!🚶‍♂️
[23:52]::INFO::✅ 

[23:54]::ERROR::Ошибка отправки команды: АСО не отвечает
[23:54]::ERROR::Ошибка отправки команды: АСО не отвечает
[23:54]::ERROR::Ошибка отправки команды: АСО не отвечает
[23:54]::ERROR::Ошибка отправки команды: АСО не отвечает
[23:54]::ERROR::Ошибка отправки команды: АСО не отвечает
[23:54]::ERROR::Ошибка отправки команды: АСО не отвечает
[23:54]::ERROR::Ошибка отправки команды: АСО не отвечает
[23:54]::ERROR::Ошибка отправки команды: АСО не отвечает
[23:54]::ERROR::Ошибка отправки команды: АСО не отвечает
[23:54]::ERROR::Ошибка отправки команды: АСО не отвечает
[23:54]::ERROR::Ошибка отправки команды: АСО не отвечает
[23:54]::ERROR::Ошибка отправки команды: АСО не отвечает
[23:54]::ERROR::Ошибка отправки команды: АСО не отвечает
[23:54]::ERROR::Ошибка отправки команды: АСО не отвечает
[23:54]::ERROR::Ошибка отправки команды: АСО не отвечает
[23:54]::ERROR::Ошибка отправки команды: АСО не отвечает
[23:54]::ERROR::Ошибка отправки команды: АСО не отвечает
[23:54]::ERROR::Ошибка отправки